# Manual 05: Optimización Básica para logistica
## Universidad del Norte
### Economía Matemática
El problema de **transporte** es un tipo de problema de optimización lineal que busca minimizar el costo de transportar bienes desde varios puntos de oferta a varios puntos de demanda. En este manual, aprenderás cómo resolver este problema utilizando la biblioteca `pulp` en Python.

### Caso de Fábrica

Una empresa de distribución de granos tiene tres silos (Silo 1, Silo 2 y Silo 3) y cuatro molinos (Molino 1, Molino 2, Molino 3 y Molino 4). La empresa necesita transportar granos desde los silos hasta los molinos de manera que se **minimicen los costos de transporte**. Cada silo tiene una cantidad limitada de granos disponibles, y cada molino tiene una demanda específica de granos que debe ser satisfecha.

#### Oferta de los Silos:
* Silo 1: 15 unidades de granos
* Silo 2: 25 unidades de granos
* Silo 3: 10 unidades de granos

#### Demanda de los Molinos:
* Molino 1: 5 unidades de granos
* Molino 2: 15 unidades de granos
* Molino 3: 15 unidades de granos
* Molino 4: 15 unidades de granos

#### Costos de Transporte (en unidades monetarias por unidad de grano):
Desde Silo 1:
* A Molino 1: 10
* A Molino 2: 2
* A Molino 3: 20
* A Molino 4: 11
        
Desde Silo 2:
* A Molino 1: 7
* A Molino 2: 9
* A Molino 3: 24
* A Molino 4: 18

Desde Silo 3:
* A Molino 1: 4
* A Molino 2: 14
* A Molino 3: 16
* A Molino 4: 18

#### Objetivo:

Minimizar el **costo total** de transporte de granos desde los silos hasta los molinos, asegurando que la oferta de cada silo no se exceda y que la demanda de cada molino se satisfaga.

### Ecuaciones del problema

$$\text{Costo total}= \sum\limits_{i\in Silos} \sum \limits_{j \in Molinos} x_{ij} \times C_{ij}$$

Las restricciones van en el orden de:

$$\sum \limits_{j \in Molinos} x_{ij} \leq \text{Oferta del Silo}_i \; \forall i \in \text{silos}$$

$$\sum \limits_{i \in Silos} x_{ij} \geq \text{Demanda de Molinos}_j \; \forall j \in \text{Molinos}$$

In [1]:
# Paquete a usar
import pulp

La idea es ir montando un diccionario de variables

In [2]:
oferta = {"Silo 1": 15, "Silo 2": 25, "Silo 3": 10}
demanda = {"Molino 1": 5, "Molino 2": 15, "Molino 3": 15, "Molino 4": 15}
costos = {"Silo 1": {"Molino 1": 10, "Molino 2": 2, "Molino 3": 20, "Molino 4": 11},
          "Silo 2": {"Molino 1": 7, "Molino 2": 9, "Molino 3": 24, "Molino 4": 18},
          "Silo 3": {"Molino 1": 4, "Molino 2": 14, "Molino 3": 16, "Molino 4": 18}}

Se define el problema central

In [4]:
problema = pulp.LpProblem("Transporte", pulp.LpMinimize)

Tenemos una de las restricciones como lo es la oferta

In [5]:
x = pulp.LpVariable.dicts("x", ((i, j) for i in oferta for j in demanda), lowBound=0, cat="Integer")

A continuación viene la parte del costo de transporte

In [6]:
problema += pulp.lpSum([x[(i, j)] * costos[i][j] for i in oferta for j in demanda]), "Costo total de transporte"

Una restricción mas como lo es la oferta

In [ ]:
for i in oferta:
    problema += pulp.lpSum([x[(i, j)] for j in demanda]) <= oferta[i], f"Oferta de {i}"

El punto de recepción de demanda al límite de lo solicitado

In [7]:
for j in demanda:
    problema += pulp.lpSum([x[(i, j)] for i in oferta]) >= demanda[j], f"Demanda de {j}"

In [8]:
problema.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/y7/xnq2gn1x2cs79dhd19wnj9wm0000gn/T/a2b5c7af320445bb88b2fcc553f2a6cf-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/y7/xnq2gn1x2cs79dhd19wnj9wm0000gn/T/a2b5c7af320445bb88b2fcc553f2a6cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 58 RHS
At line 63 BOUNDS
At line 76 ENDATA
Problem MODEL has 4 rows, 12 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 455 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 455 to -1.79769e+308
Probing was tried 0 times and 

1

In [9]:
if problema.status == pulp.LpStatusOptimal:
    costo_minimo = pulp.value(problema.objective)
    print(f"El costo total de transporte mínimo es: {costo_minimo}")
    for i in oferta:
        for j in demanda:
            cantidad = x[(i, j)].varValue
            if cantidad > 0:
                print(f"Se deben transportar {cantidad} camiones cargados desde {i} hasta {j}.")
else:
    print("El problema no tiene solución óptima.")

El costo total de transporte mínimo es: 455.0
Se deben transportar 15.0 camiones cargados desde Silo 1 hasta Molino 2.
Se deben transportar 15.0 camiones cargados desde Silo 1 hasta Molino 4.
Se deben transportar 5.0 camiones cargados desde Silo 3 hasta Molino 1.
Se deben transportar 15.0 camiones cargados desde Silo 3 hasta Molino 3.


Este manual te ha guiado a través de los pasos necesarios para resolver un problema de transporte utilizando la biblioteca pulp en Python. Puedes ajustar los datos de oferta, demanda y costos para explorar diferentes escenarios y ver cómo afectan los resultados.

-------

# Ejercicio Final
### Optimización de Costos de Transporte  
#### Contexto  

Una empresa tiene tres fábricas (**A, B, C**) y cuatro centros de distribución (**1, 2, 3, 4**).  
Cada fábrica tiene una **capacidad de producción** limitada y cada centro de distribución tiene una **demanda específica**.  
El objetivo de la empresa es **minimizar los costos de transporte** desde las fábricas hasta los centros de distribución.  

## Datos  

### Capacidades de las fábricas  
- **Fábrica A**: 500 unidades  
- **Fábrica B**: 600 unidades  
- **Fábrica C**: 400 unidades  

### Demandas de los centros de distribución  
- **Centro 1**: 300 unidades  
- **Centro 2**: 400 unidades  
- **Centro 3**: 500 unidades  
- **Centro 4**: 300 unidades  


| Fábricas    | Centro 1  | Centro 2  | Centro 3  | Centro 4  |
|------------|-----------|-----------|-----------|-----------|
| *Fábrica A*  | 1000000 | 1200000 | 1500000 | 1300000 |
| *Fábrica B*  | 1100000 | 1000000 | 1400000 | 1200000 |
| *Fábrica C*  | 1300000 | 1200000 | 700000  | 900000  |


## Objetivo  

Minimizar el **costo total de transporte** cumpliendo con las **capacidades de las fábricas** y las **demandas de los centros de distribución**.